In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, metrics, Sequential
from tensorflow import keras

In [6]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [ ]:
tf.random.set_seed(2345)

In [71]:
(x,y),(x_val,y_val) = datasets.mnist.load_data()

In [72]:
x.shape

(60000, 28, 28)

In [104]:
conv_layers = [ #5 units of conv + max pooling
    # unit 1
    #kernel 的数量
    layers.Conv2D(64, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.Conv2D(64, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2,2], strides = 2,padding = 'same'),
    # unit 2
    layers.Conv2D(128, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.Conv2D(128, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2,2], strides = 2,padding = 'same'),
    # unit 3
    layers.Conv2D(256, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.Conv2D(256, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2,2], strides = 2,padding = 'same'),
    # unit 4
    layers.Conv2D(512, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.Conv2D(512, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2,2], strides = 2,padding = 'same'),
    # unit 5
    layers.Conv2D(512, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.Conv2D(512, kernel_size = [3,3],padding = 'same', activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2,2], strides = 2,padding = 'same')
]

In [101]:
conv_net = Sequential(conv_layers)
conv_net.build(input_shape=[None,28,28,1])
# conv_net.summary()
#[b,28,28,1] -> [b,1,1,512]

In [75]:
fc_net = Sequential([
    layers.Dense(256,activation = tf.nn.relu),
    layers.Dense(128,activation = tf.nn.relu),
    layers.Dense(64,activation = tf.nn.relu),
    layers.Dense(10,activation = None),
])
fc_net.build(input_shape=[None,512])
optimizer = optimizers.Adam(learning_rate=1e-3)

In [76]:
x = tf.reshape(x,[-1 ,28,28,1])

In [77]:
def preprocess(x,y):
    x = tf.cast(x,dtype = tf.float32) / 255.
    y = tf.cast(y,dtype= tf.int32)
    return x,y    

In [78]:
train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(preprocess).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_val,y_val))
test_db = train_db.map(preprocess).batch(128)

In [79]:
sample = next(iter(train_db))

In [80]:
sample[0].shape

TensorShape([128, 28, 28, 1])

In [81]:
sample[1].shape

TensorShape([128])

In [82]:
variables = conv_net.trainable_variables +fc_net.trainable_variables
# [1,2] +[3,4] = [1,2,3,4] 
for epoch in range(20):
    for step,(x,y) in enumerate(train_db):
        with tf.GradientTape() as tape:
#             [b,28,28,1] -> [b,1,1,512]
            out = conv_net(x)
            out = tf.reshape(out,[-1,512])
#             [b,512] -> [b,100]
            logits = fc_net(out)
            y_onehot = tf.one_hot(y,depth = 10)
            loss = tf.losses.categorical_crossentropy(y_onehot,logits,from_logits= True)
            loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss,variables)
        optimizer.apply_gradients(zip(grads,variables))
        if step % 100 ==0:
            print(epoch,step,'loss',loss)
    #
    total_num = 0
    total_corrext = 0
    for x,y in test_db:
        out = conv_net(x)
        out =tf.reshape(out,[-1,512])
        logits =fc_net(out)
        pro = tf.nn.softmax(logits,axis =1)
        pred = tf.argmax(prob,axis =1)
        pred = tf.cast(pred,dtype=tf.int32)
        correct = tf.cast(tf.equal(pred,y),dtype= tf.int32)
        correct = tf.reduce_sum(correct)
        total_correct += int(correct)
        total_num += x.shape[0]
    acc = total_correct / total_num
    print(epoch,'acc',acc)
    

0 0 loss tf.Tensor(2.302414, shape=(), dtype=float32)


KeyboardInterrupt: 